In [ ]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

template = """You are an AI language model assistant. Your task is to generate 5 different versions of
    the given user questions to retrieve relevant data from the internet. In order to this well, you will need to 
    think like a political consultant incharge of running a campaign. By generating multiple perspectives on the user question, 
    your goal is to help the user get a better answer with a full picture. Provide these alternative questions separated by newlines. 
    Original question: {question}"""
decomp_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
query_analyzer = (decomp_prompt | llm | StrOutputParser() | (lambda x: x.split("\n")))
sub_queries = query_analyzer.invoke({"question":question})
sub_queries

## Define Agents

In [ ]:
# import crewai

# define an agent
    # duckduckgo
    # wikipedia agent
    # web scraper focused on newspapers

# for loop
# each query 
# crew = Crew(
#   agents=[researcher, manager],
#   tasks=[task1, task2],
#   process=Process.sequential,
#   verbose=2
# )
# result = crew.kickoff() 

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.task import TaskOutput
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from crewai_tools import (
    WebsiteSearchTool,
    ScrapeWebsiteTool
)

search_tool = DuckDuckGoSearchRun()
# wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# wikidata_tool = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())
website_search_tool = WebsiteSearchTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
# Create agents
search_agent = Agent(
    role='Researcher',
    goal='Find up-to-date information and news articles',
    backstory="""An expert researcher that is able to search the web to find
        information on anything. You don't give up until all the research is complete.""",
    tools=[search_tool, website_search_tool],
    # verbose=True
)

In [ ]:
# Create agents
scraper_agent = Agent(
    role='Scraper',
    goal='Find the most relevant websites related to search query.',
    backstory="""An expert web-scraper, you don't stop trying until you get the information you are looking for. If you fail trying to scrape a website, you don't give up.""",
    tools=[scrape_tool],
    # verbose=True
)

In [ ]:
# Create agents
writer = Agent(
    role='Report & Memo Writer',
    goal='With the provided information, generate a long and thorough report.',
    backstory="""You are a Political Campaign manager with excellent writing skills. Given all the research data, you are able to summarize, draw conclusions, and then generate a report for the rest of the team to understand the question.""",
    tools=[scrape_tool],
    # verbose=True
)

In [ ]:
query_task_list = []

for query in sub_queries:
    # iterate through queries, define search and scrape task
    search_task = Task(
        description=f"""Search the web to gather information on {query} using trusted sources such as 
        """,
        expected_output="A list of facts",
        agent= search_agent,
        tools=[search_tool, website_search_tool]
    )

    scrape_task = Task(
        description="""Take the data provided and write a detailed report to help an election campaign come up with a plan to win.  
            """,
        agent=scraper_agent,
        expected_output="A report on the city",
        tools=[scrape_tool],
    )
    # once tasks are defined for each query, append to task_list
    query_task_list.append(search_task, scrape_task)


In [ ]:
# Create final task for summary/report generation

In [ ]:
# create the crew
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[], # add in the agents
  tasks=[], #add in the tasks 
  process=Process.sequential,
  verbose=2
)
result = crew.kickoff()

![test]("./76E82B5C-6B16-4566-8554-6B1B7D4C33C7_1_105_c.jpeg")

![rag-flowchart-langchain](rag-flowchart.png)



Steps:
1. Get query
2. Expand Query using Decomposition
3. Pass all sub-queries to crew ai agents
4. Review all info
5. create a memo

Agents
1. scraper
2. search duck duck go
3. Compiler/Reviewer - request more info from agents 1 and 2?
4. Memo Writer

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
openai_model = os.getenv("OPENAI_MODEL_NAME")
os.environ["OPENAI_MODEL_NAME"]= openai_model
os.environ["OPENAI_API_KEY"]= openai_key

In [ ]:
question = "What is the political landscape of Gainesville, Fl?"

## Decomposition Step